In [1]:
# Import Dependencies
import numpy as np
import pandas as pd
import pyspark as spark
import sklearn as skl
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Create pandas dataframe from the csv file
df = pd.read_csv('Pseo_All.csv')

In [4]:
df.head()

,agg_level_pseo,inst_level,institution,degree_level,cip_level,cipcode,grad_cohort,grad_cohort_years,geo_level,geography,...,status_y1_earnings,status_y5_earnings,y1_grads_emp,y1_grads_emp_instate,y5_grads_emp,y5_grads_emp_instate,status_y1_grads_emp,status_y1_grads_emp_instate,status_y5_grads_emp,status_y5_grads_emp_instate
0,38,I,105100,5,A,0,0,3,N,0,...,1,1,49897.0,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0
1,38,I,105100,5,A,0,0,3,N,0,...,1,1,49897.0,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0
2,38,I,105100,5,A,0,0,3,N,0,...,1,1,49897.0,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0
3,38,I,105100,5,A,0,0,3,N,0,...,1,1,49897.0,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0
4,38,I,105100,5,A,0,0,3,N,0,...,1,1,49897.0,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0


In [5]:
df.dtypes

agg_level_pseo                   int64
inst_level                      object
institution                      int64
degree_level                     int64
cip_level                       object
cipcode                          int64
grad_cohort                      int64
grad_cohort_years                int64
geo_level                       object
geography                        int64
ind_level                       object
industry                         int64
y1_p50_earnings                  int64
y5_p50_earnings                  int64
y1_ipeds_count                   int64
y5_ipeds_count                   int64
status_y1_earnings               int64
status_y5_earnings               int64
y1_grads_emp                   float64
y1_grads_emp_instate           float64
y5_grads_emp                   float64
y5_grads_emp_instate           float64
status_y1_grads_emp            float64
status_y1_grads_emp_instate    float64
status_y5_grads_emp            float64
status_y5_grads_emp_insta

In [6]:
df.groupby('inst_level').inst_level.count()

inst_level
I    223160
Name: inst_level, dtype: int64

In [7]:
df.groupby('geo_level').geo_level.count()

geo_level
N    223160
Name: geo_level, dtype: int64

In [8]:
df.groupby('cip_level').cip_level.count()

cip_level
2    97984
4    26872
2    58122
4    17994
A    22188
Name: cip_level, dtype: int64

In [9]:
df.groupby('ind_level').ind_level.count()

ind_level
A    223160
Name: ind_level, dtype: int64

In [10]:
# Since inst_level, geo_level, and ind level are all the same value they are dropped from the dataframe
dropped_df= df.drop(['inst_level', 'geo_level', 'ind_level'], axis = 1)
dropped_df.head()

,agg_level_pseo,institution,degree_level,cip_level,cipcode,grad_cohort,grad_cohort_years,geography,industry,y1_p50_earnings,...,status_y1_earnings,status_y5_earnings,y1_grads_emp,y1_grads_emp_instate,y5_grads_emp,y5_grads_emp_instate,status_y1_grads_emp,status_y1_grads_emp_instate,status_y5_grads_emp,status_y5_grads_emp_instate
0,38,105100,5,A,0,0,3,0,0,37267,...,1,1,49897.0,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0
1,38,105100,5,A,0,0,3,0,0,37267,...,1,1,49897.0,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0
2,38,105100,5,A,0,0,3,0,0,37267,...,1,1,49897.0,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0
3,38,105100,5,A,0,0,3,0,0,37267,...,1,1,49897.0,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0
4,38,105100,5,A,0,0,3,0,0,37267,...,1,1,49897.0,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0


In [11]:
#Replacing all "A" values with the number 1 so that they are all the same data type
dropped_df.loc[dropped_df["cip_level"] == "A", "cip_level"] = 1

# Changing data type
dropped_df['cip_level']= dropped_df['cip_level'].astype(int)
dropped_df.dtypes

agg_level_pseo                   int64
institution                      int64
degree_level                     int64
cip_level                        int64
cipcode                          int64
grad_cohort                      int64
grad_cohort_years                int64
geography                        int64
industry                         int64
y1_p50_earnings                  int64
y5_p50_earnings                  int64
y1_ipeds_count                   int64
y5_ipeds_count                   int64
status_y1_earnings               int64
status_y5_earnings               int64
y1_grads_emp                   float64
y1_grads_emp_instate           float64
y5_grads_emp                   float64
y5_grads_emp_instate           float64
status_y1_grads_emp            float64
status_y1_grads_emp_instate    float64
status_y5_grads_emp            float64
status_y5_grads_emp_instate    float64
dtype: object

In [12]:
# One hot encoding is used on cip_level column

# Create the OneHotEncoder instance
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)

# Fit the encoder and produce encoded DataFrame
encode_df = pd.DataFrame(enc.fit_transform(dropped_df.cip_level.values.reshape(-1,1)))

# Rename encoded columns
encode_df.columns = enc.get_feature_names(['cip_level'])
encode_df.head()

,cip_level_1,cip_level_2,cip_level_4
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0


In [13]:
# Merge the two DataFrames together and drop the cip_level column
dropped_df.merge(encode_df,left_index=True,right_index=True).drop("cip_level",1)

,agg_level_pseo,institution,degree_level,cipcode,grad_cohort,grad_cohort_years,geography,industry,y1_p50_earnings,y5_p50_earnings,...,y1_grads_emp_instate,y5_grads_emp,y5_grads_emp_instate,status_y1_grads_emp,status_y1_grads_emp_instate,status_y5_grads_emp,status_y5_grads_emp_instate,cip_level_1,cip_level_2,cip_level_4
0,38,105100,5,0,0,3,0,0,37267,51849,...,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
1,38,105100,5,0,0,3,0,0,37267,51849,...,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
2,38,105100,5,0,0,3,0,0,37267,51849,...,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
3,38,105100,5,0,0,3,0,0,37267,51849,...,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
4,38,105100,5,0,0,3,0,0,37267,51849,...,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223155,48,3730300,3,52,2001,5,0,0,26840,35645,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0
223156,48,3789400,3,24,2001,5,0,0,26167,39475,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0
223157,48,3789400,3,24,2001,5,0,0,26167,39475,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0
223158,48,3956300,3,13,2001,5,0,0,29553,31238,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0


In [14]:
dropped_df.columns

Index(['agg_level_pseo', 'institution', 'degree_level', 'cip_level', 'cipcode',
       'grad_cohort', 'grad_cohort_years', 'geography', 'industry',
       'y1_p50_earnings', 'y5_p50_earnings', 'y1_ipeds_count',
       'y5_ipeds_count', 'status_y1_earnings', 'status_y5_earnings',
       'y1_grads_emp', 'y1_grads_emp_instate', 'y5_grads_emp',
       'y5_grads_emp_instate', 'status_y1_grads_emp',
       'status_y1_grads_emp_instate', 'status_y5_grads_emp',
       'status_y5_grads_emp_instate'],
      dtype='object')

In [15]:
# Applying the condition
dropped_df["y1_p50_earnings"] = np.where(df["y1_p50_earnings"] < 35000, 0, 1)
dropped_df.groupby('y1_p50_earnings').y1_p50_earnings.count()

y1_p50_earnings
0    114374
1    108786
Name: y1_p50_earnings, dtype: int64

In [16]:
# Define features dataset
X= dropped_df.drop(['y1_p50_earnings','y5_p50_earnings', 'y1_ipeds_count', 'status_y1_earnings','status_y5_earnings','y1_grads_emp','y1_grads_emp_instate', 'y5_grads_emp',
       'y5_grads_emp_instate', 'status_y1_grads_emp', 'status_y1_grads_emp_instate', 'status_y5_grads_emp','y5_ipeds_count','status_y5_grads_emp_instate'],axis= 1)
X.head()


,agg_level_pseo,institution,degree_level,cip_level,cipcode,grad_cohort,grad_cohort_years,geography,industry
0,38,105100,5,1,0,0,3,0,0
1,38,105100,5,1,0,0,3,0,0
2,38,105100,5,1,0,0,3,0,0
3,38,105100,5,1,0,0,3,0,0
4,38,105100,5,1,0,0,3,0,0


In [17]:
# Define target dataset
y = dropped_df["y1_p50_earnings"].ravel()
y[:5]

array([1, 1, 1, 1, 1])

In [18]:
# Splitting into Train and Test sets for Logistic regression.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, stratify=y)

In [19]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',
   max_iter=200,
   random_state=78)

In [20]:
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=78)

In [21]:
y_pred = classifier.predict(X_test)

In [22]:
# Ovserve accuracy score from Logistic Regression
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.5305789568022943


In [113]:
# Splitting into Train and Test sets.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [114]:
# Create scaler instance
X_scaler = skl.preprocessing.StandardScaler()

# Fit the scaler
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [116]:
# Define the model - deep neural net
import tensorflow as tf

# Create the Keras Sequential model
nn_model = tf.keras.models.Sequential()

# Add our first Dense layer, including the input layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="relu", input_dim=9))

# Add the output layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nn_model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 1)                 10        
                                                                 
 dense_25 (Dense)            (None, 1)                 2         
                                                                 
Total params: 12
Trainable params: 12
Non-trainable params: 0
_________________________________________________________________


In [117]:
# Compile the model
nn_model.compile(optimizer="adam", loss="mean_squared_error", metrics=["accuracy"])

In [118]:
# Fit the model to the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=40)

Epoch 1/40
5231/5231 [==============================] - 9s 2ms/step - loss: 0.2307 - accuracy: 0.6040
Epoch 2/40
5231/5231 [==============================] - 8s 2ms/step - loss: 0.2234 - accuracy: 0.6253
Epoch 3/40
5231/5231 [==============================] - 8s 2ms/step - loss: 0.2229 - accuracy: 0.6269
Epoch 4/40
5231/5231 [==============================] - 8s 2ms/step - loss: 0.2227 - accuracy: 0.6294
Epoch 5/40
5231/5231 [==============================] - 8s 2ms/step - loss: 0.2226 - accuracy: 0.6298
Epoch 6/40
5231/5231 [==============================] - 8s 2ms/step - loss: 0.2225 - accuracy: 0.6310
Epoch 7/40
5231/5231 [==============================] - 9s 2ms/step - loss: 0.2225 - accuracy: 0.6323
Epoch 8/40
5231/5231 [==============================] - 8s 2ms/step - loss: 0.2225 - accuracy: 0.6328
Epoch 9/40
5231/5231 [==============================] - 8s 2ms/step - loss: 0.2225 - accuracy: 0.6330
Epoch 10/40
5231/5231 [==============================] - 8s 1ms/step - loss: 0.222

In [119]:
# Prepare for test on year 5 earnings
dropped_df["y5_p50_earnings"] = np.where(df["y5_p50_earnings"] < 50000, 0, 1)
dropped_df.groupby('y5_p50_earnings').y1_p50_earnings.count()

y5_p50_earnings
0    116852
1    106308
Name: y1_p50_earnings, dtype: int64

In [120]:
# Define features dataset
X= dropped_df.drop(['y1_p50_earnings','y5_p50_earnings', 'y1_ipeds_count', 'status_y1_earnings','status_y5_earnings','y1_grads_emp','y1_grads_emp_instate', 'y5_grads_emp',
       'y5_grads_emp_instate', 'status_y1_grads_emp', 'status_y1_grads_emp_instate', 'status_y5_grads_emp','y5_ipeds_count','status_y5_grads_emp_instate'],axis= 1)
X.head()


,agg_level_pseo,institution,degree_level,cip_level,cipcode,grad_cohort,grad_cohort_years,geography,industry
0,38,105100,5,1,0,0,3,0,0
1,38,105100,5,1,0,0,3,0,0
2,38,105100,5,1,0,0,3,0,0
3,38,105100,5,1,0,0,3,0,0
4,38,105100,5,1,0,0,3,0,0


In [121]:
# Define target dataset
y = dropped_df["y5_p50_earnings"].ravel()
y[:5]

array([1, 1, 1, 1, 1])

In [122]:
# Splitting into Train and Test sets.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [123]:
# Create scaler instance
X_scaler = skl.preprocessing.StandardScaler()

# Fit the scaler
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [125]:
# Define the model - deep neural net
import tensorflow as tf

# Create the Keras Sequential model
nn_model = tf.keras.models.Sequential()

# Add our first Dense layer, including the input layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="relu", input_dim=9))

# Add the output layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nn_model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 1)                 10        
                                                                 
 dense_29 (Dense)            (None, 1)                 2         
                                                                 
Total params: 12
Trainable params: 12
Non-trainable params: 0
_________________________________________________________________


In [126]:
# Compile the model
nn_model.compile(optimizer="adam", loss="mean_squared_error", metrics=["accuracy"])

In [127]:
# Fit the model to the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=40)

Epoch 1/40
5231/5231 [==============================] - 8s 1ms/step - loss: 0.2519 - accuracy: 0.5346
Epoch 2/40
5231/5231 [==============================] - 7s 1ms/step - loss: 0.2449 - accuracy: 0.5580
Epoch 3/40
5231/5231 [==============================] - 7s 1ms/step - loss: 0.2418 - accuracy: 0.5785
Epoch 4/40
5231/5231 [==============================] - 7s 1ms/step - loss: 0.2320 - accuracy: 0.6091
Epoch 5/40
5231/5231 [==============================] - 7s 1ms/step - loss: 0.2278 - accuracy: 0.6010
Epoch 6/40
5231/5231 [==============================] - 12s 2ms/step - loss: 0.2266 - accuracy: 0.6018
Epoch 7/40
5231/5231 [==============================] - 9s 2ms/step - loss: 0.2262 - accuracy: 0.6020
Epoch 8/40
5231/5231 [==============================] - 8s 2ms/step - loss: 0.2260 - accuracy: 0.6020
Epoch 9/40
5231/5231 [==============================] - 8s 1ms/step - loss: 0.2259 - accuracy: 0.6024
Epoch 10/40
5231/5231 [==============================] - 8s 1ms/step - loss: 0.22